In [1]:
try:
    import google.colab
    USE_COLAB = True
except:
    USE_COLAB = False

RUN_TESTS = True

if USE_COLAB:
    print("Don't forget to avoid disconnections:")
    print("""
    function ClickConnect(){
        console.log("Clicking"); 
        document.querySelector("colab-connect-button").click() 
    }
    setInterval(ClickConnect,60000)
    
    """)

Don't forget to avoid disconnections:

    function ClickConnect(){
        console.log("Clicking"); 
        document.querySelector("colab-connect-button").click() 
    }
    setInterval(ClickConnect,60000)
    
    


In [2]:
# in google colab uncomment this
if USE_COLAB:
    import os

    os.system('apt-get install -y xvfb')
    os.system('wget https://raw.githubusercontent.com/yandexdataschool/Practical_DL/fall18/xvfb -O ../xvfb')

    file_format = 'https://raw.githubusercontent.com/yandexdataschool/Practical_RL/spring20/week06_policy_based/{file} -O {file}'

    os.system('wget ' + file_format.format(file='atari_wrappers.py'))
    os.system('wget ' + file_format.format(file='env_batch.py'))
    os.system('wget ' + file_format.format(file='runners.py'))

    os.system('apt-get install -y python-opengl ffmpeg')
    os.system('pip install pyglet==1.2.4')

    os.system('python -m pip install -U pygame --user')

    print('setup complete')

    # # XVFB will be launched if you run on a server
    # if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    #     !bash ../xvfb start
    #     os.environ['DISPLAY'] = ':1'

setup complete


# Implementing Advantage-Actor Critic (A2C)

In this notebook you will implement Advantage Actor Critic algorithm that trains on a batch of Atari 2600 environments running in parallel. 

Firstly, we will use environment wrappers implemented in file `atari_wrappers.py`. These wrappers preprocess observations (resize, grayscal, take max between frames, skip frames and stack them together) and rewards. Some of the wrappers help to reset the environment and pass `done` flag equal to `True` when agent dies.
File `env_batch.py` includes implementation of `ParallelEnvBatch` class that allows to run multiple environments in parallel. To create an environment we can use `nature_dqn_env` function. Note that if you are using 
PyTorch and not using `tensorboardX` you will need to implement a wrapper that will log **raw** total rewards that the *unwrapped* environment returns and redefine the implemention of `nature_dqn_env` function here. 



In [3]:
if USE_COLAB:
    %tensorflow_version 1.x
import numpy as np
import math
import torch
import torch.nn as nn
from atari_wrappers import nature_dqn_env
from atari_wrappers import NumpySummaries
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm import trange
from pandas import DataFrame

test_env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=8, summaries='Numpy')
test_obs = test_env.reset()
assert test_obs.shape == (8, 84, 84, 4)
assert test_obs.dtype == np.uint8

TensorFlow 1.x selected.


Next, we will need to implement a model that predicts logits and values. It is suggested that you use the same model as in [Nature DQN paper](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) with a modification that instead of having a single output layer, it will have two output layers taking as input the output of the last hidden layer. **Note** that this model is different from the model you used in homework where you implemented DQN. You can use your favorite deep learning framework here. We suggest that you use orthogonal initialization with parameter $\sqrt{2}$ for kernels and initialize biases with zeros. 

In [0]:
def init_weights(m):
    if type(m) in (nn.Linear, nn.Conv2d):
        nn.init.orthogonal_(m.weight.data, math.sqrt(2.0))
        if m.bias is not None:
            m.bias.data.fill_(0)
            
def conv2d_size_out(size, kernel_size, stride):
    """
    common use case:
    cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
    cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    to understand the shape for dense layer's input
    """
    size = np.array(size)
    return (size - (kernel_size - 1) - 1) // stride  + 1

In [0]:
class Agent(nn.Module):
    def __init__(self, state_shape, n_actions):
        """
        An actor-critic agent
        param: state_shape shape of a single test with shape [h, w, n_channels]
        """
        super().__init__()
        
        self.n_actions = n_actions
        n_features = 512
        
        self.body_network = self.get_body_network(state_shape, n_features).apply(init_weights)
        self.v_head = nn.Linear(n_features, 1).apply(init_weights)
        self.logits_head = nn.Linear(n_features, n_actions).apply(init_weights)
        self.buffer = None

    def set_buffer(self, buffer):
        self.buffer = buffer

    def get_buffer(self):
        return self.buffer

    def get_body_network(self, in_shape, out_features):
        h, w, in_channels = in_shape
        network = nn.Sequential()
        
        h, w = conv2d_size_out([h, w], 8, 4)
        network.add_module('Conv1', nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=8, stride=4))
        network.add_module('Act1', nn.ReLU())

        h, w = conv2d_size_out([h, w], 4, 2)
        network.add_module('Conv2', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2))
        network.add_module('Act2', nn.ReLU())
        
        h, w = conv2d_size_out([h, w], 3, 1)
        network.add_module('Conv3', nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1))
        network.add_module('Act3', nn.ReLU())
        network.add_module('flatten', nn.Flatten())
        
        network.add_module('FC1', nn.Linear(in_features=h * w * 64, out_features=out_features))
        network.add_module('Act4', nn.ReLU())
        return network

    def forward(self, state):
        """
        takes agent's observation, returns logits and state value
        :param state: a batch of 4-frame buffers, shape = [batch_size, h, w, n_channels]
        """

        state_t = state.permute(0, 3, 1, 2)
        state_t = state_t.float() / 255.
        state_t = self.body_network(state_t)
        logits = self.logits_head(state_t)
        state_value = self.v_head(state_t)
        
        return logits, state_value
    
    def get_actions(self, state):
        logits, state_value = self.forward(state)
        probas = nn.functional.softmax(logits).cpu().detach()
        
        return torch.multinomial(probas, num_samples=1), logits, state_value

**Test**

In [6]:
if RUN_TESTS:
    test_state = torch.ones((2, 64, 64, 4))
    test_n_actions = 4
    test_agent = Agent(test_state.shape[1:], test_n_actions)

    test_logits, test_state_value = test_agent(test_state)

    assert test_logits.shape == torch.Size((test_state.shape[0], test_n_actions))
    assert test_state_value.shape == torch.Size((test_state.shape[0], 1))
    print('Test passed')

Test passed


You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a dictionary of all the arrays that are needed to interact with an environment and train the model.
 Note that actions must be an `np.ndarray` while the other
tensors need to have the type determined by your deep learning framework. 

In [0]:
class Policy:
    def __init__(self, model, device=torch.device('cpu')):
        self.model = model
        self.device = device
    
    def act(self, inputs):
        """
        Implementation of policy which calls model, samples actions and computes their log probs
        :return dict containing keys ['actions', 'logits', 'log_probs', 'values'].
        """
        
        inputs = torch.FloatTensor(inputs).to(self.device)
        logits, state_values = self.model.forward(inputs)

        probas = nn.functional.softmax(logits, -1)
        log_probs = nn.functional.log_softmax(logits, -1)
        actions = torch.multinomial(probas, num_samples=1).cpu().detach().numpy().flatten()
        return {
            'actions': actions,
            'logits': logits,
            'probas': probas,
            'full_log_probas': log_probs,
            'log_probs': log_probs[np.arange(len(actions)), actions],
            'values': state_values.view(-1)
        }

**Test**

In [8]:
if RUN_TESTS:
    test_state = np.ones((2, 64, 64, 4))
    test_n_actions = 4
    test_agent = Agent(test_state.shape[1:], test_n_actions)
    test_policy = Policy(test_agent)

    test_res_dict = test_policy.act(test_state)

    assert all(nn.functional.log_softmax(test_res_dict['logits'], -1)[np.arange(2), test_res_dict['actions']] == test_res_dict['log_probs'])
    print("Test passed")

Test passed


Next will pass the environment and policy to a runner that collects partial trajectories from the environment. 
The class that does is is already implemented for you.

In [0]:
from runners import EnvRunner

This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys 

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment of specified length $T$ &mdash; the size of partial trajectory. 

**Test** Runner

In [10]:
if RUN_TESTS:
    test_agent = Agent(test_obs.shape[1:], test_env.action_space.n)
    test_trajectory = EnvRunner(test_env, Policy(test_agent), 3).get_next()
    print(test_trajectory.keys())
    print(test_trajectory['actions'])
    print(test_trajectory['values'])
    print(test_trajectory['rewards'])
    print(test_trajectory['resets'])

dict_keys(['actions', 'logits', 'probas', 'full_log_probas', 'log_probs', 'values', 'observations', 'rewards', 'resets', 'state'])
[array([4, 5, 0, 4, 4, 2, 2, 4]), array([1, 3, 0, 1, 4, 4, 2, 5]), array([3, 0, 1, 1, 3, 4, 3, 2])]
[tensor([-0.2773, -0.2773, -0.2773, -0.2773, -0.2773, -0.2773, -0.2773, -0.2773],
       grad_fn=<ViewBackward>), tensor([-0.2773, -0.2773, -0.2773, -0.2773, -0.2773, -0.2773, -0.2773, -0.2773],
       grad_fn=<ViewBackward>), tensor([-0.2751, -0.2751, -0.2751, -0.2751, -0.2751, -0.2751, -0.2751, -0.2751],
       grad_fn=<ViewBackward>)]
[array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0.])]
[array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False]), array([False, False, False, False, False, False, False, False])]


To train the part of the model that predicts state values you will need to compute the value targets. 
Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected. 
Thus, we can implement and use `ComputeValueTargets` callable. 
The formula for the value targets is simple:

$$
\hat v(s_t) = \sum_{t'=t}^{T - 1}\gamma^{t'-t}r_{t'}\prod_{t''=t}^{t'} resets_{t''}{} + resets_{T-1}\gamma^{T-1-t} \hat{v}(s_{T-1}),
$$

In implementation, however, do not forget to use 
`trajectory['resets']` flags to check if you need to add the value targets at the next step when 
computing value targets for the current step. You can access `trajectory['state']['latest_observation']`
to get last observations in partial trajectory &mdash; $s_{t+T}$.

**Test**

In [0]:
class ComputeValueTargets:
    def __init__(self, policy, gamma=0.99):
        self.policy = policy
        self.gamma = gamma
    
    def __call__(self, trajectory):
        # This method should modify trajectory inplace by adding 
        # an item with key 'value_targets' to it. 
        resets = trajectory['resets']
        rewards = trajectory['rewards']

        curr_value = self.policy.act(trajectory['state']['latest_observation'])['values']
        target_values = []
        for r, done in reversed(list(zip(rewards, resets))):
            r = torch.tensor(r, device=curr_value.device)
            done = torch.tensor(done, device=curr_value.device)
            curr_value = torch.where(
                done,
                curr_value*self.gamma + r,
                r
            )
            target_values.append(curr_value)
        trajectory['value_targets'] = target_values[::-1]
        return trajectory

**Test**

In [12]:
if RUN_TESTS:
    test_obs = test_env.reset()
    test_n_actions = test_env.action_space.n
    test_agent = Agent(test_obs.shape[1:], test_n_actions)
    test_policy = Policy(test_agent)
    test_compute_value_targets = ComputeValueTargets(test_policy)

    test_trajectory = EnvRunner(test_env, test_policy, 2).get_next()
    assert len(test_compute_value_targets(test_trajectory)['value_targets']) == 2
    assert len(test_compute_value_targets(test_trajectory)['value_targets'][0]) == 8 
    print("Test passed")

Test passed


After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `T * nenvs`, i.e. you essentially need
to flatten the first two dimensions. 

**Test**

In [0]:
class MergeTimeBatch:
    """ Merges first two axes typically representing time and env batch. """
    def __call__(self, trajectory):
        for key, val in trajectory.items():
            if key in ['actions', 'observations', 'rewards', 'resets']:
                val = torch.tensor(val)
            elif key in ['probas', 'full_log_probas', 'logits', 'log_probs', 'values', 'value_targets']:
                val = torch.stack(val)
            else:
                continue

            val = val.reshape(-1, *val.shape[2:])
            trajectory[key] = val
        return trajectory

**Test**

In [14]:
if RUN_TESTS:
    test_obs = test_env.reset()
    test_n_actions = test_env.action_space.n
    test_agent = Agent(test_obs.shape[1:], test_n_actions)
    test_policy = Policy(test_agent)
    test_merge_time_batch = MergeTimeBatch()
    test_trajectory = ComputeValueTargets(test_policy)(EnvRunner(test_env, test_policy, 2).get_next())

    test_res = test_merge_time_batch(test_trajectory)
    for key in ['actions', 'observations', 'rewards', 'resets', 'logits', 'log_probs', 'values', 'value_targets']:
        assert test_res[key].shape[0] == 16

    print("Test passed")

Test passed


Now is the time to implement the advantage actor critic algorithm itself. You can look into your lecture,
[Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and [lecture](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) by Sergey Levine.

In [0]:
class A2C:
    def __init__(
        self,
        policy,
        optimizer,
        value_loss_coef=0.005,
        entropy_coef=0.002,
        max_grad_norm=0.5
    ):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm
    
    def policy_loss(self, trajectory):
        # You will need to compute advantages here. 
        advantages = trajectory['value_targets'] - trajectory['values']
        return -torch.mean(trajectory['log_probs'] * advantages.detach())
    
    def value_loss(self, trajectory):
        advantages = trajectory['value_targets'].detach() - trajectory['values']
        value_loss = advantages.pow(2).mean()
        return value_loss
    
    def entropy_loss(self, trajectory):
        entropy = torch.sum(trajectory['probas'] * trajectory['full_log_probas'], -1).mean()
        return entropy
    
    def loss(self, trajectory):
        value_loss = self.value_loss(trajectory)
        policy_loss = self.policy_loss(trajectory)
        entropy_loss = self.entropy_loss(trajectory)
        
        return self.value_loss_coef*value_loss + policy_loss + self.entropy_coef*entropy_loss, value_loss, policy_loss, entropy_loss

    def step(self, trajectory):
        loss, value_loss, policy_loss, entropy_loss = self.loss(trajectory)
        
        loss.backward()
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), self.max_grad_norm)
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss, value_loss, policy_loss, entropy_loss, grad_norm


Now you can train your model. With reasonable hyperparameters training on a single GTX1080 for 10 million steps across all batched environments (which translates to about 5 hours of wall clock time)
it should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last 
episodes in each environment in the batch) of about 600. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with all environments. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages
* A2C loss

For optimization we suggest you use RMSProp with learning rate starting from 7e-4 and linearly decayed to 0, smoothing constant (alpha in PyTorch and decay in TensorFlow) equal to 0.99 and epsilon equal to 1e-5.

**Define Parameters**

In [0]:
N_ENVS = 8
T_STEP = 5

USE_CUDA = True
LAST_EPOCH = 0
TOTAL_EPOCH = 10000000 // N_ENVS // T_STEP

DROW_STEP = 1000
FIG_SIZE = 17, 8
SAVE_STEP = 10000

MOUNT_DIR = '/content/drive/'
MODEL_MASK = (MOUNT_DIR + 'My Drive/models/rl/a2c/' if USE_COLAB else 'models/') + 'a2c_{last_epoch}_epoch'

**Mount GDrive**

In [17]:
if USE_COLAB:
    from google.colab import files, drive
    drive.mount(MOUNT_DIR)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


**Select Device**

In [18]:
if USE_CUDA and torch.cuda.is_available():
    DEVICE_ID = 0
    device = torch.device('cuda:%d' % DEVICE_ID)
else:
    device = torch.device('cpu')
print("device: ", device)

device:  cuda:0


**Set up env**

In [0]:
env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=N_ENVS, summaries='Numpy')
env.clear()
obs = env.reset()

**define model**

In [20]:
if LAST_EPOCH:
    model = torch.load(MODEL_MASK.format(last_epoch=LAST_EPOCH), map_location=torch.device('cpu'))
else:
    model = Agent(obs.shape[1:], env.action_space.n)
    model.set_buffer([[], [], [], [], [], []])
model.to(device)

Agent(
  (body_network): Sequential(
    (Conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (Act1): ReLU()
    (Conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (Act2): ReLU()
    (Conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (Act3): ReLU()
    (flatten): Flatten()
    (FC1): Linear(in_features=3136, out_features=512, bias=True)
    (Act4): ReLU()
  )
  (v_head): Linear(in_features=512, out_features=1, bias=True)
  (logits_head): Linear(in_features=512, out_features=6, bias=True)
)

**define stuff**

In [0]:
policy = Policy(model, device)
runner = EnvRunner(
    env,
    policy,
    nsteps=T_STEP,
    transforms=[
                ComputeValueTargets(policy),
                MergeTimeBatch()
                ]
)
optimizer = torch.optim.RMSprop(model.parameters(), lr=7e-4, alpha=0.99, eps=1e-5)
a2c = A2C(policy, optimizer)

In [0]:
def moving_average(x, span=100): return DataFrame(
    {'x': np.asarray(x)}).x.ewm(span=span).mean().values

**learning**

In [0]:
%%time
curves_names = ['A2c loss', 'value_loss', 'policy_loss', 'enropy_loss', 'grad_norm', 'Reward']
curves = model.get_buffer()
ylims = [[0, 300], [0, 300], [-2, 2], [-2, 2], [0, 100], [-1, 800]]

for i in trange(LAST_EPOCH, TOTAL_EPOCH):
    trajectory = runner.get_next()
    losses = a2c.step(trajectory)
    
    for j, loss in enumerate(losses):
        curves[j].append(loss)

    mean_reward = env.get_values("SpaceInvadersNoFrameskip-v4/reward_mean_100") or [(None, 0)]
    curves[-1].append(mean_reward[-1][1])
    if mean_reward[-1][1] > 600:
        print("TASK PASSED!")

    if i and i % DROW_STEP == 0:
        clear_output(True)
        fig1, axs = plt.subplots(3, 2, figsize=FIG_SIZE, constrained_layout=True)
        for j, curve in enumerate(curves):
            axs[j // 2][j % 2].set_ylim(ylims[j])
            axs[j // 2][j % 2].set_title(curves_names[j])
            try:
                smoth_curve = moving_average(curve, span=100) if j != 5 else curve
            except:
                print(f"Error with {curves_names[j]}")
                print(curve)
                continue
            axs[j // 2][j % 2].plot(smoth_curve, label=f'{curves_names[j]}= {smoth_curve[-1]}')
        fig1.legend()
        plt.show()
    
    if i != LAST_EPOCH and i % SAVE_STEP == 0:
        model.set_buffer(curves)
        torch.save(model, MODEL_MASK.format(last_epoch=i))


  0%|          | 430/250000 [02:41<25:54:12,  2.68it/s]